##Data observation and analysis

In [ ]:
import pandas as pd

df = pd.read_csv("/content/dataset_challenge1_ai.csv")

print(df.shape)
print(df.columns)
print(df.head())
print(df.tail())
print(df.info())
print(df.describe())


##Descriptive statistical indicators

In [ ]:
# For study hours
print("STUDY HOURS PER WEEK:")
print("MEAN:", df["study_hours_per_week"].mean())
print("MEDIAN:", df["study_hours_per_week"].median())
print("MAX:", df["study_hours_per_week"].max())
print("Q1:", df["study_hours_per_week"].quantile(0.25))
print("---")

# For sleep hours
print("SLEEP HOURS PER DAY:")
print("MEAN:", df["sleep_hours_per_day"].mean())
print("MEDIAN:", df["sleep_hours_per_day"].median())
print("MAX:", df["sleep_hours_per_day"].max())
print("Q1:", df["sleep_hours_per_day"].quantile(0.25))
print("---")

# For attendance percentage
print("ATTENDANCE PERCENTAGE:")
print("MEAN:", df["attendance_percentage"].mean())
print("MEDIAN:", df["attendance_percentage"].median())
print("MAX:", df["attendance_percentage"].max())
print("Q1:", df["attendance_percentage"].quantile(0.25))
print("---")

# For assignments
print("ASSIGNMENTS COMPLETED:")
print("MEAN:", df["assignments_completed"].mean())
print("MEDIAN:", df["assignments_completed"].median())
print("MAX:", df["assignments_completed"].max())
print("Q1:", df["assignments_completed"].quantile(0.25))
print("---")

# For final grade
print("FINAL GRADE:")
print("MEAN:", df["final_grade"].mean())
print("MEDIAN:", df["final_grade"].median())
print("MAX:", df["final_grade"].max())
print("Q1:", df["final_grade"].quantile(0.25))


##Replacing NULL and invalid data

Our Null data only relates to the parents' education and we could have left out all the replacement values, but given that new data may have blanks in these values, these replacements were made to avoid errors in the future.

In [ ]:
# Replace missing numeric values with median
df["study_hours_per_week"].fillna(df["study_hours_per_week"].median(), inplace=True)
df["sleep_hours_per_day"].fillna(df["sleep_hours_per_day"].median(), inplace=True)
df["attendance_percentage"].fillna(df["attendance_percentage"].median(), inplace=True)
df["assignments_completed"].fillna(df["assignments_completed"].median(), inplace=True)
df["final_grade"].fillna(df["final_grade"].median(), inplace=True)

# Fill missing categorical values with "Unknown"
df["parental_education"].fillna("Unknown", inplace=True)
df["participation_level"].fillna("Unknown", inplace=True)

# Fill missing yes/no columns with most frequent value
most_common_internet = df["internet_access"].value_counts().index[0]
df["internet_access"].fillna(most_common_internet, inplace=True)
most_common_extra = df["extracurricular"].value_counts().index[0]
df["extracurricular"].fillna(most_common_extra, inplace=True)
most_common_job = df["part_time_job"].value_counts().index[0]
df["part_time_job"].fillna(most_common_job, inplace=True)

# Fix unreasonable values
df.loc[df["sleep_hours_per_day"] < 2, "sleep_hours_per_day"] = 2
df.loc[df["study_hours_per_week"] < 1, "study_hours_per_week"] = 1
df.loc[df["attendance_percentage"] < 0.1, "attendance_percentage"] = 0.1

# Final check
print("Data cleaning completed!")
print("Number of rows/columns:", df.shape)
print("Missing values:")
print(df.isnull().sum())


##Pie chart for people with jobs

In [ ]:
import matplotlib.pyplot as plt

# Pie chart for participation_level
counts = df["participation_level"].value_counts()
values = counts.values
labels = counts.index

colors = ["lightpink", "lightblue", "lightgreen"]

plt.figure(figsize=(8, 6))
plt.pie(values, labels=labels, colors=colors, autopct="%1.1f%%")
plt.title("Student Participation Level")
plt.show()

# Pie chart for part_time_job
counts_job = df["part_time_job"].value_counts()
values_job = counts_job.values
labels_job = counts_job.index

colors_job = ["lightcoral", "lightgreen"]

plt.figure(figsize=(8, 6))
plt.pie(values_job, labels=labels_job, colors=colors_job, autopct="%1.1f%%")
plt.title("Having a Part-time Job")
plt.show()


##Histogram and boxplot for the three main variables

In [ ]:
# Histograms and Box Plots
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Histograms
axes[0,0].hist(df['study_hours_per_week'], bins=20, color='lightblue', edgecolor='black')
axes[0,0].set_title('Histogram - Study Hours')
axes[0,0].set_xlabel('Hours')
axes[0,0].set_ylabel('Count')

axes[0,1].hist(df['sleep_hours_per_day'], bins=15, color='lightgreen', edgecolor='black')
axes[0,1].set_title('Histogram - Sleep Hours')
axes[0,1].set_xlabel('Hours')

axes[0,2].hist(df['final_grade'], bins=20, color='lightcoral', edgecolor='black')
axes[0,2].set_title('Histogram - Final Grade')
axes[0,2].set_xlabel('Grade')

# Box Plots
axes[1,0].boxplot(df['study_hours_per_week'])
axes[1,0].set_title('Box Plot - Study Hours')

axes[1,1].boxplot(df['sleep_hours_per_day'])
axes[1,1].set_title('Box Plot - Sleep Hours')

axes[1,2].boxplot(df['final_grade'])
axes[1,2].set_title('Box Plot - Final Grade')

plt.tight_layout()
plt.show()


In [ ]:
## Correlation chart

Here we find that students who studied more hours per week and completed more assignments received higher final grades.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Calculate correlation matrix
numeric_columns = ['study_hours_per_week', 'sleep_hours_per_day',
                  'attendance_percentage', 'assignments_completed',
                  'final_grade']

correlation_matrix = df[numeric_columns].corr()

# Heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', center=0)
plt.title('Correlation Matrix')
plt.show()

# Identify features most correlated with final_grade
print("Correlation of all features with Final Grade:")
correlation_with_grade = correlation_matrix['final_grade'].drop('final_grade').sort_values(ascending=False)

for feature, corr in correlation_with_grade.items():
    print(f"{feature}: {corr:.3f}")

# Top 2 features with strongest correlation
print("\nTop 2 Features Most Correlated with Final Grade:")
top_2_features = correlation_with_grade.head(2)
for feature, corr in top_2_features.items():
    print(f"{feature}: {corr:.3f}")


##Discriminant histogram based on performance

In [ ]:
import matplotlib.pyplot as plt

# Average grade
avg_grade = df["final_grade"].mean()
print(f"🌡️ Average final grade: {avg_grade:.2f}")

# Use mean ± 1 std for thresholds
std_grade = df["final_grade"].std()
upper_threshold = avg_grade + std_grade/2
lower_threshold = avg_grade - std_grade/2

print(f"📊 Upper threshold: {upper_threshold:.2f}")
print(f"📊 Lower threshold: {lower_threshold:.2f}")

# Classification with mean
df["performance"] = ["successful" if grade >= avg_grade else "less_successful" for grade in df["final_grade"]]

print(f"✅ Selected threshold: {avg_grade:.2f}")

# Simple histograms
plt.figure(figsize=(15, 12))

# 1. Study hours
plt.subplot(2, 2, 1)
success_study = df[df["performance"] == "successful"]["study_hours_per_week"]
less_study = df[df["performance"] == "less_successful"]["study_hours_per_week"]
plt.hist(success_study, alpha=0.7, label="Successful", color="green", bins=10)
plt.hist(less_study, alpha=0.7, label="Less Successful", color="red", bins=10)
plt.xlabel("Study Hours")
plt.ylabel("Count")
plt.legend()
plt.title("Study Hours")

# 2. Sleep hours
plt.subplot(2, 2, 2)
success_sleep = df[df["performance"] == "successful"]["sleep_hours_per_day"]
less_sleep = df[df["performance"] == "less_successful"]["sleep_hours_per_day"]
plt.hist(success_sleep, alpha=0.7, label="Successful", color="green", bins=10)
plt.hist(less_sleep, alpha=0.7, label="Less Successful", color="red", bins=10)
plt.xlabel("Sleep Hours")
plt.ylabel("Count")
plt.legend()
plt.title("Sleep Hours")

# 3. Attendance
plt.subplot(2, 2, 3)
success_attendance = df[df["performance"] == "successful"]["attendance_percentage"]
less_attendance = df[df["performance"] == "less_successful"]["attendance_percentage"]
plt.hist(success_attendance, alpha=0.7, label="Successful", color="green", bins=10)
plt.hist(less_attendance, alpha=0.7, label="Less Successful", color="red", bins=10)
plt.xlabel("Attendance %")
plt.ylabel("Count")
plt.legend()
plt.title("Class Attendance")

# 4. Assignments
plt.subplot(2, 2, 4)
success_assign = df[df["performance"] == "successful"]["assignments_completed"]
less_assign = df[df["performance"] == "less_successful"]["assignments_completed"]
plt.hist(success_assign, alpha=0.7, label="Successful", color="green", bins=10)
plt.hist(less_assign, alpha=0.7, label="Less Successful", color="red", bins=10)
plt.xlabel("Assignments Completed")
plt.ylabel("Count")
plt.legend()
plt.title("Assignments Completed")

plt.tight_layout()
plt.show()

# Final counts
success_count = len(df[df["performance"] == "successful"])
less_count = len(df[df["performance"] == "less_successful"])
print(f"🎯 Successful students: {success_count}")
print(f"🎯 Less successful students: {less_count}")


As can be seen, when we set the boundary between successful and less successful people at 70, the number of successful and less successful people becomes approximately equal and close to each other. Therefore, the graph moves almost close to each other, so we need to check to make sure that the differences are correct and that our data is not problematic.

In [ ]:
# Compare feature averages between classes
print("📊 Average feature values for each class:")

features = ["study_hours_per_week", "sleep_hours_per_day",
            "attendance_percentage", "assignments_completed"]

for feature in features:
    mean_success = df[df["performance"] == "successful"][feature].mean()
    mean_less = df[df["performance"] == "less_successful"][feature].mean()
    diff = mean_success - mean_less

    print(f"\n{feature}:")
    print(f"   Successful: {mean_success:.2f}")
    print(f"   Less Successful: {mean_less:.2f}")
    print(f"   Difference: {diff:+.2f} ({'✅ higher for successful' if diff > 0 else '❌ higher for less successful'})")


##Scatter chart

In [ ]:
Looking at this graph, we see that study hours and sleep hours are the best options for modeling and the data is separated very well. On the contrary, in the student participation graph, the data indentation is too much and does not seem like a good option at all.

In [ ]:
import matplotlib.pyplot as plt

# Scatter plots for 3 feature combinations
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# 1. Study Hours vs Final Grade
success = df[df["performance"] == "successful"]
less = df[df["performance"] == "less_successful"]

axes[0].scatter(success["study_hours_per_week"], success["final_grade"],
               color="green", alpha=0.7, label="Successful")
axes[0].scatter(less["study_hours_per_week"], less["final_grade"],
               color="red", alpha=0.7, label="Less Successful")
axes[0].set_xlabel("Study Hours Per Week")
axes[0].set_ylabel("Final Grade")
axes[0].set_title("Study Hours vs Final Grade")
axes[0].legend()

# 2. Study Hours vs Attendance
axes[1].scatter(success["study_hours_per_week"], success["attendance_percentage"],
               color="green", alpha=0.7, label="Successful")
axes[1].scatter(less["study_hours_per_week"], less["attendance_percentage"],
               color="red", alpha=0.7, label="Less Successful")
axes[1].set_xlabel("Study Hours Per Week")
axes[1].set_ylabel("Attendance Percentage")
axes[1].set_title("Study Hours vs Attendance")
axes[1].legend()

# 3. Sleep Hours vs Final Grade
axes[2].scatter(success["sleep_hours_per_day"], success["final_grade"],
               color="green", alpha=0.7, label="Successful")
axes[2].scatter(less["sleep_hours_per_day"], less["final_grade"],
               color="red", alpha=0.7, label="Less Successful")
axes[2].set_xlabel("Sleep Hours Per Day")
axes[2].set_ylabel("Final Grade")
axes[2].set_title("Sleep Hours vs Final Grade")
axes[2].legend()

plt.tight_layout()
plt.show()


In [ ]:
##Modeling with simple rules

In [ ]:
# Simple IF-ELSE model
def predict_performance(study_hours, sleep_hours, attendance, assignments):
    """
    Simple rule-based model to predict 'successful' or 'less_successful'
    """
    score = 0

    # Rule 1: Study hours
    if study_hours > 40:
        score += 3
    elif study_hours > 30:
        score += 2
    elif study_hours > 20:
        score += 1

    # Rule 2: Sleep hours
    if 6 <= sleep_hours <= 8:  # standard sleep
        score += 2
    elif sleep_hours > 8:      # oversleeping
        score += 1
    elif sleep_hours < 4:      # very low sleep
        score -= 1

    # Rule 3: Attendance
    if attendance > 0.8:
        score += 2
    elif attendance > 0.6:
        score += 1

    # Rule 4: Assignments
    if assignments > 0.7:
        score += 2
    elif assignments > 0.4:
        score += 1

    # Final prediction
    if score >= 5:
        return "successful"
    else:
        return "less_successful"

# Test model on first student
test_student = df.iloc[0]
result = predict_performance(
    test_student["study_hours_per_week"],
    test_student["sleep_hours_per_day"],
    test_student["attendance_percentage"],
    test_student["assignments_completed"]
)

print(f"Prediction for student 0: {result}")
print(f"Actual: {test_student['performance']}")


In [ ]:
# Evaluate model on all students
correct_predictions = 0
total_students = len(df)

for i in range(total_students):
    student = df.iloc[i]
    prediction = predict_performance(
        student["study_hours_per_week"],
        student["sleep_hours_per_day"],
        student["attendance_percentage"],
        student["assignments_completed"]
    )

    if prediction == student["performance"]:
        correct_predictions += 1

accuracy = (correct_predictions / total_students) * 100

print(f"Correct predictions: {correct_predictions}")
print(f"Incorrect predictions: {total_students - correct_predictions}")
print(f"Model Accuracy: {accuracy:.2f}%")

# Check distribution of classes
success_count = len(df[df["performance"] == "successful"])
less_count = len(df[df["performance"] == "less_successful"])

print(f"\nNumber of successful students: {success_count}")
print(f"Number of less successful students: {less_count}")


In [ ]:
print("✅ IF-ELSE model achieved 72.8% accuracy!")
print("   - Good for quick and simple predictions")
print("   - Interpretable and transparent")
print("   - But more advanced models are needed for higher accuracy")


##Final modeling

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score

# Linear Regression model for predicting final_grade
X = df[["study_hours_per_week", "sleep_hours_per_day",
        "attendance_percentage", "assignments_completed"]]
y = df["final_grade"]

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = LinearRegression()
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluation
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("=== LINEAR REGRESSION MODEL RESULTS ===")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"R² Score: {r2:.2f}")
print("=" * 40)

# Compare predictions with actual
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.7)
plt.plot([y.min(), y.max()], [y.min(), y.max()], 'r--', lw=2)
plt.xlabel('Actual Final Grade')
plt.ylabel('Predicted Final Grade')
plt.title('Predicted vs Actual - Linear Regression')
plt.show()
